In [1]:
import base64, requests, sys#, os
# import config as c
import PureCloudPlatformClientV2
from PureCloudPlatformClientV2.rest import ApiException
from pprint import pprint
import logging
import json
# from IPython.display import display
import pandas as pd
# import csv
from datetime import datetime,date, timedelta
from six import iteritems
from pathlib import Path
import re
# import sqlite3
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
# fernet import lookup on google for encryption

In [2]:

print("---------------------------------------------------")
print("- Genesys Cloud Python Client Credentials Example -")
print("---------------------------------------------------")

# CLIENT_ID = os.environ['GENESYS_CLOUD_CLIENT_ID']
# CLIENT_SECRET = os.environ['GENESYS_CLOUD_CLIENT_SECRET']
# ENVIRONMENT = os.environ['GENESYS_CLOUD_ENVIRONMENT'] # eg. mypurecloud.com

CLIENT_ID = 'c2108a92-3db9-4b58-a51b-6e985e9effe7'
CLIENT_SECRET = 'EaBXNtpRgHHXh_STUBSE9CTDFr-Ku-57fIy3DOCNUn8'
ENVIRONMENT = 'usw2.pure.cloud'

region = PureCloudPlatformClientV2.PureCloudRegionHosts.us_west_2
PureCloudPlatformClientV2.configuration.host = region.get_api_host()

# Base64 encode the client ID and client secret
authorization = base64.b64encode(bytes(CLIENT_ID + ":" + CLIENT_SECRET, "ISO-8859-1")).decode("ascii")

# Prepare for POST /oauth/token request
request_headers = {
    "Authorization": f"Basic {authorization}",
    "Content-Type": "application/x-www-form-urlencoded"
}
request_body = {
    "grant_type": "client_credentials"
}

---------------------------------------------------
- Genesys Cloud Python Client Credentials Example -
---------------------------------------------------


In [3]:
# Get token
response = requests.post(f"https://login.{ENVIRONMENT}/oauth/token", data=request_body, headers=request_headers)

# Check response
if response.status_code == 200:
    print("Got token")
else:
    print(f"Failure: { str(response.status_code) } - { response.reason }")
    sys.exit(response.status_code)

# Get JSON response body
response_json = response.json()

# Prepare for GET /api/v2/authorization/roles request
requestHeaders = {
    "Authorization": f"{ response_json['token_type'] } { response_json['access_token']}"
}

# Get roles
response = requests.get(f"https://api.{ENVIRONMENT}/api/v2/authorization/roles", headers=requestHeaders)

# Check response
if response.status_code == 200:
    print("Got roles")
else:
    print(f"Failure: { str(response.status_code) } - { response.reason }")
    sys.exit(response.status_code)

# Print roles
print("\nRoles:")
for entity in response.json()["entities"]:
    print(f"  { entity['name'] }")

print("\nDone")

Got token
Got roles

Roles:
  BSS Agent test
  AI Agent
  BSS Agent no WFM
  BSS Agent Assist
  BSS Digicomm agent
  BSS Agent Gamification
  BSS Outbound Agent
  BSS Agent
  Communicate - User
  Email Ninja
  BSS Group Workspaces Admin
  BSS Temp admin
  BSS Trainer - Monitor
  BSS Trainer - Monitor and Coach
  BSS WFM Supervisor
  BSS Workforce Management - Admin
  BSS Supervisor
  BSS outsource Supervisor
  CCOM Account Manager
  Co-browse Agent
  BSS Quality Administrator
  BSS Self Activate in Queues
  Developer
  admin
  Communicate - Admin

Done


In [5]:
# ========================================================================================================================================================================================
# ==================================================================== Conversation Aggregation Query ====================================================================================
# Genesys API Explorer: /api/v2/analytics/conversations/aggregates/query
# Parameters:
#   Interval: 2023-01-02T00:00:00/2023-01-03T00:00:00
#   Group By: queueId
#   Filter:
#       Type: and
#       Predicates:
#           Type: dimension
#           Dimension: mediaType
#           Operator: matches
#           Value: voice
#   Metrics: nOffered, nTransferred, tAcw, tConnected
# Created By: Clayton Ayers
# Created Date: 1/24/2023
# Modified Date: NA

# Change this to pull previous day for nightly run
# May need to ask Ken how to configure datetime.now() to date.now() so that it pulls yesterday starting at 12am instead of what time this process will run in the future
# Also need to figure out how to pull in all time dates with BU Stats and group by Date/Interval since beginning of time
def conversation_aggregates_get_interval_string():
    # Gets an ISO-8601 interval from now for the last x days
    now = datetime.now().replace(microsecond=0)
    week_ago = (now - timedelta(days=1)).replace(microsecond=0)
    return f"{ week_ago.isoformat() }/{ now.isoformat() }"


print("----------------------------------------------------------------------------------")
print("- Querying Que Historical Statistics for Conversation Aggregate Query - ")
print("----------------------------------------------------------------------------------")

# Credentials set above
# Environment set above
# OAuth when using Client Credentials set above
api_client = PureCloudPlatformClientV2.api_client.ApiClient() \
             .get_client_credentials_token(CLIENT_ID, CLIENT_SECRET)

# Genesys Cloud Objects
routing_api = PureCloudPlatformClientV2.RoutingApi(api_client)
analytics_api = PureCloudPlatformClientV2.AnalyticsApi(api_client)

# Get "Support" queue by name
try: 
    queue_data = routing_api.get_routing_queues(name='CI Customer Service') #test was Support, test CI Customer Service
except ApiException as e:
    print("Exception when calling RoutingApi->get_routing_quest: {e}")
    sys.exit()

print(f"queueData: {queue_data}")


----------------------------------------------------------------------------------
- Querying Que Historical Statistics for Conversation Aggregate Query - 
----------------------------------------------------------------------------------
queueData: {'entities': [{'acw_settings': {'timeout_ms': 30000,
                                'wrapup_prompt': 'MANDATORY_TIMEOUT'},
               'agent_owned_routing': {'enable_agent_owned_callbacks': False,
                                       'max_owned_callback_delay_hours': 2,
                                       'max_owned_callback_hours': 2},
               'auto_answer_only': True,
               'bullseye': None,
               'calling_party_name': 'Cigars International',
               'calling_party_number': '+18882442790',
               'created_by': 'a54a216b-78ee-436a-923b-f26c82faed75',
               'date_created': datetime.datetime(2022, 3, 10, 15, 33, 7, 987000, tzinfo=tzutc()),
               'date_modified': datetime.dat

In [6]:
# Store queue ID
queue_id = queue_data.entities[0].id

# Build analytics query
query = PureCloudPlatformClientV2.ConversationAggregationQuery()
# query.interval = conversation_aggregates_get_interval_string()
query.interval = '2023-03-13T00:00:00/2023-03-19T11:59:00'
query.granularity = 'P1D'
# query.granularity = 'PT1H' #To get results by hour per day

query.group_by = ['queueid', 'requestedRoutingSkillId']

query.metrics = ['nOffered', 'nOutbound', 'nConnected', 'nOutboundAbandoned', 'nOutboundAttempted', 'nOutboundConnected', 'nOverSla', 'nTransferred', 'nConsult', 'nConsultTransferred', 'tAcd', 'tAnswered', 'tConnected', 'tHandle', 'tHeld', 'tWait']

# query.filter = PureCloudPlatformClientV2.ConversationAggregateQueryFilter()
# query.filter.type = 'and'
# query.filter.clauses = [PureCloudPlatformClientV2.ConversationAggregateQueryClause()]

# query.filter.clauses[0].type = 'or'
# query.filter.clauses[0].predicates = [PureCloudPlatformClientV2.ConversationAggregateQueryPredicate()]
# query.filter.clauses[0].predicates[0].dimension = 'queueId'
# query.filter.clauses[0].predicates[0].value = queue_id


print(f"query: {query}")

# Execute analytics query
query_result = analytics_api.post_analytics_conversations_aggregates_query(query)

# pprint(f"queryResult: {query_result}")


print(query_result)
query_result_test=json.loads(query_result.to_json())
print("- Querying Que Historical Statistics for Conversation Aggregate Query Results! - ")
pprint(query_result_test)

query: {'alternate_time_dimension': None,
 'filter': None,
 'flatten_multivalued_dimensions': None,
 'granularity': 'P1D',
 'group_by': ['queueid', 'requestedRoutingSkillId'],
 'interval': '2023-03-13T00:00:00/2023-03-19T11:59:00',
 'metrics': ['nOffered',
             'nOutbound',
             'nConnected',
             'nOutboundAbandoned',
             'nOutboundAttempted',
             'nOutboundConnected',
             'nOverSla',
             'nTransferred',
             'nConsult',
             'nConsultTransferred',
             'tAcd',
             'tAnswered',
             'tConnected',
             'tHandle',
             'tHeld',
             'tWait'],
 'time_zone': None,
 'views': None}


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



{'results': [{'data': [{'interval': '2023-03-13T00:00:00.000Z/2023-03-14T00:00:00.000Z',
                        'metrics': [{'metric': 'nConnected',
                                     'qualifier': None,
                                     'stats': {'count': 24,
                                               'count_negative': None,
                                               'count_positive': None,
                                               'current': None,
                                               'denominator': None,
                                               'max': None,
                                               'min': None,
                                               'numerator': None,
                                               'p95': None,
                                               'p99': None,
                                               'ratio': None,
                                               'sum': None,
                                

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
# ============ This is the DataFrame to use to pull back Metric:Stats for each record, will need to nest this within the outer DataFrame ======================================================
test_list = query_result_test['results']


test_rec2 = query_result_test['results']

dict = {}
dfStats = pd.DataFrame(dict)

for i in range(len(test_rec2)):
    x = query_result_test['results'][i]['group']
    if 'requestedRoutingSkillId' in x:
        requestedRoutingSkillId = query_result_test['results'][i]['group']['requestedRoutingSkillId']
    else:
        requestedRoutingSkillId = 'No Skill Required'
    if 'queueId' in x:
        queueIdCheck = query_result_test['results'][i]['group']['queueId']
        if queueIdCheck is not None:
            RequestedRoutingSkillId = requestedRoutingSkillId
            queueId = query_result_test['results'][i]['group']['queueId']
            mediaType = query_result_test['results'][i]['group']['mediaType']
            for k in range(len(query_result_test['results'][i]['data'])):
                interval = query_result_test['results'][i]['data'][k]['interval']
                for l in range(len(query_result_test['results'][i]['data'][k]['metrics'])):
                    metric = query_result_test['results'][i]['data'][k]['metrics'][l]['metric']
                    mediaTypeCount = query_result_test['results'][i]['data'][k]['metrics'][l]['stats']['count']
                    mediaTypeMax = query_result_test['results'][i]['data'][k]['metrics'][l]['stats']['max']
                    mediaTypeMin = query_result_test['results'][i]['data'][k]['metrics'][l]['stats']['min']
                    mediaTypeSum = query_result_test['results'][i]['data'][k]['metrics'][l]['stats']['sum']
                    dfStatsv2 = {'queueId':queueId,'mediaType':mediaType,'requestedRoutingSkillId':RequestedRoutingSkillId,'interval':interval,'metric':metric,'mediaTypeCount':mediaTypeCount,'mediaTypeMax':mediaTypeMax,'mediaTypeMin':mediaTypeMin,'mediaTypeSum':mediaTypeSum}
                    print(dfStatsv2)
                    dfStats = pd.concat([dfStats,dfStatsv2],ignore_index=True)
dfStats = dfStats.sort_values(by=['queueId'])
dfStats = dfStats.reset_index(drop=True)

dfStats[['startDate', 'endDate']] = dfStats['interval'].str.split('/', expand=True)
dfStats[['startDate', 'endDate']] = dfStats[['startDate', 'endDate']].apply(pd.to_datetime)
dfStats = dfStats.drop(columns=['interval'])


{'queueId': '5f4b9981-88f8-4941-9172-40d15aaf1235', 'mediaType': 'callback', 'requestedRoutingSkillId': 'No Skill Required', 'interval': '2023-03-13T00:00:00.000Z/2023-03-14T00:00:00.000Z', 'metric': 'nConnected', 'mediaTypeCount': 2, 'mediaTypeMax': None, 'mediaTypeMin': None, 'mediaTypeSum': None}


TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [8]:
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif(x == 'float64'):
            dataList.append('float')
        elif(x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList
columnDataType = getColumnDtypes(dfStats.dtypes)

# print(dfStats)

# USE apply method to change column datatypes in dfStats to import into SQL
dfStats[['mediaTypeMax','mediaTypeMin','mediaTypeSum']] = dfStats[['mediaTypeMax','mediaTypeMin','mediaTypeSum']].apply(pd.to_numeric)
dfStats['startDate'] = pd.to_datetime(dfStats.startDate).dt.tz_localize(None)
dfStats['endDate'] = pd.to_datetime(dfStats.endDate).dt.tz_localize(None)
# print(dfStats)
# rename count,max,min,sum to mediatypeCount etc
# Agg by week by queueID

In [ ]:
server = 'EUSVMSQL01.ci826.local'
database = 'AnalyticsTeamDB'

connection = create_engine('mssql+pyodbc://EUSVMSQL01.ci826.local/AnalyticsTeamDB?driver=SQL+Server+Native+Client+11.0')

#dfStats.to_sql(name='ccConversationsAgg', con=connection, if_exists='append',index=False, schema='Stage')